In [1]:
import pandas as pd
import random

# Ouvrir le fichier d'entrée en mode lecture
def create_lang_df(path, langue):
    with open(path, 'r', encoding='utf-8') as fichier_entree:
        lignes = fichier_entree.readlines()
    lignes_propres = [ligne.strip() for ligne in lignes]

    df=pd.DataFrame(data=lignes_propres, columns=['phrase'])
    df['langue'] = langue
    return df


df_fr = create_lang_df('../data/small_vocab_fr','fr')
df_fr = df_fr.sample(frac=1, random_state=42)
df_en = create_lang_df('../data/small_vocab_en','en')
df_en = df_en.sample(frac=1, random_state=24)
n_rows = len(df_fr)
print('Nombre de lignes de small_vocab:',n_rows)


Nombre de lignes de small_vocab: 137860


In [2]:
# Créez un vocabulaire complet à partir des données anglaises et française
from transformers import BertTokenizerFast
from collections import Counter

df = pd.concat([df_fr['phrase'],df_en['phrase']])
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')  
vocab = Counter(token for doc in df for token in tokenizer.tokenize(doc))
len(vocab)

581

In [3]:
# créer 2 dataframes: 1 train (70% des phrases) et 1 test (30% des phrases) avec, dans chaque dataframe, 
# 50% des phrases en anglais et 50% en français

n_train = int(n_rows*0.7)
df_train = pd.concat([df_fr.iloc[:n_train],df_en.iloc[:n_train]]).sample(frac=1, random_state=42).reset_index(drop=True)
df_test = pd.concat([df_fr.iloc[n_train:],df_en.iloc[n_train:]]).sample(frac=1, random_state=24).reset_index(drop=True)
pd.set_option('display.max_colwidth', 150)
display(df_train)

,phrase,langue
0,"france is dry during march , and it is never wonderful in winter .",en
1,"their least favorite fruit is the grape , but his least favorite is the mango .",en
2,"elle déteste les poires , les fraises et les citrons .",fr
3,she drove that little blue automobile .,en
4,"ils n'aiment les citrons , les pommes et le pamplemousse .",fr
...,...,...
192999,"he dislikes grapes , lemons , and bananas.",en
193000,"china is sometimes rainy during summer , but it is usually dry in autumn .",en
193001,"india is never pleasant during april , and it is usually beautiful in winter .",en
193002,"new jersey is usually quiet during fall , but it is usually warm in june .",en


In [4]:
import numpy as np

# Créez un DataFrame BOW avec les phrases (lignes) et les fréquences de chaque token (colonnes)
def create_BOW(data):
    BOW = []
    for ligne in data:
        l_tokenised = tokenizer.tokenize(ligne)
        BOW.append([l_tokenised.count(token) for token in vocab])
    return BOW

X_train = create_BOW(df_train['phrase'])
y_train = df_train['langue'].values.tolist() 

#
X_test = create_BOW(df_test['phrase'])
y_test = df_test['langue'].values.tolist()


In [5]:
from sklearn import svm
from sklearn import naive_bayes

# On entraine et on prédit
clf_nb = naive_bayes.BernoulliNB()
clf_nb.fit(X_train, y_train)
y_pred_nb = clf_nb.predict(X_test)
pd.crosstab(y_test,y_pred_nb,rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,en,fr
Classe réelle,,
en,41358,0
fr,0,41358


In [6]:
from sklearn.ensemble import GradientBoostingClassifier
# Créer un classificateur clf et entraîner le modèle sur l'ensemble d'entraînement
clf_gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

# Calculer les prédictions 
y_pred_gb = clf_gb.predict(X_test)
pd.crosstab(y_test,y_pred_gb,rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,en,fr
Classe réelle,,
en,41358,0
fr,0,41358


In [7]:

# Instanciation d'un exemple
exemple = ["france is often snowy during spring , and it is relaxing in january .",
           "elle adore les voitures très luxueuses, et toi ?",
           "she loves very luxurious cars, don't you?",
           "I love you, ma chérie en janvier avec des citrons under the rain"]


# Prédiction des classes de l'exemple par le classifieur Naive Bayes
predictions_nb = clf_nb.predict(create_BOW(exemple))

# Prédiction des classes de l'exemple par le classifieur Gradiant Boosting
predictions_gb = clf_gb.predict(create_BOW(exemple))

# Affichage des prédictions
print('Prédictions Naive Bayes :',predictions_nb)
print('Prédictions Gradiant Boosting :',predictions_gb)

Prédictions Naive Bayes : ['en' 'fr' 'en' 'fr']
Prédictions Gradiant Boosting : ['en' 'fr' 'en' 'fr']
